# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dampit,-8.2116,112.7493,23.68,96,98,1.02,ID,1713742325
1,1,beibei,29.8256,106.4339,19.65,94,100,0.48,CN,1713742325
2,2,papatowai,-46.5619,169.4708,10.03,67,74,7.95,NZ,1713742323
3,3,mount gambier,-37.8333,140.7667,14.56,55,6,2.41,AU,1713742329
4,4,waitangi,-43.9535,-176.5597,14.34,87,37,3.13,NZ,1713742330


In [3]:
"https://api.openweathermap.org/data/2.5/weather?lat=57&lon=-2.15&appid={weather_api_key}&units=imperial"

'https://api.openweathermap.org/data/2.5/weather?lat=57&lon=-2.15&appid={weather_api_key}&units=imperial'

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_condition = city_data_df.loc[(city_data_df["Max Temp"] <= 90) & (city_data_df["Max Temp"] >= 70)]
ideal_condition = city_data_df.loc[city_data_df["Wind Speed"] < 10]
ideal_condition = city_data_df.loc[city_data_df["Cloudiness"] == 0]
# Drop any rows with null values
ideal_weather_df = ideal_condition.dropna()
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,keflavik,64.0049,-22.5624,4.95,93,0,2.57,IS,1713742335
16,16,iguape,-24.7081,-47.5553,20.67,87,0,2.37,BR,1713742336
20,20,sesheke,-17.4759,24.2968,17.65,46,0,2.11,ZM,1713742340
30,30,page,36.9147,-111.4558,27.05,12,0,3.13,US,1713742350
33,33,gulf park estates,30.3919,-88.7611,21.01,43,0,7.20,US,1713742354
...,...,...,...,...,...,...,...,...,...,...
577,577,brookings,44.3114,-96.7984,15.21,21,0,5.66,US,1713742638
590,590,rustaq,23.3908,57.4244,25.62,52,0,0.77,OM,1713742679
591,591,marsh harbour,26.5412,-77.0636,24.39,72,0,6.27,BS,1713742577
592,592,chipinge,-20.1883,32.6236,13.71,68,0,1.51,ZW,1713742681


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Lat", "Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,keflavik,64.0049,-22.5624,4.95,93,0,2.57,IS,1713742335,
16,iguape,-24.7081,-47.5553,20.67,87,0,2.37,BR,1713742336,
20,sesheke,-17.4759,24.2968,17.65,46,0,2.11,ZM,1713742340,
30,page,36.9147,-111.4558,27.05,12,0,3.13,US,1713742350,
33,gulf park estates,30.3919,-88.7611,21.01,43,0,7.20,US,1713742354,
35,bethel,41.3712,-73.4140,9.64,44,0,3.09,US,1713742160,
39,carnarvon,-24.8667,113.6333,22.16,44,0,7.17,AU,1713742187,
47,broken hill,-31.9500,141.4333,19.18,33,0,4.71,AU,1713742358,
50,callao,-12.0667,-77.1500,23.16,73,0,4.63,PE,1713742359,
55,sur,22.5667,59.5289,26.38,77,0,4.01,OM,1713742366,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "radius": 1000,
    "limit": 20,
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
keflavik - nearest hotel: No hotel found
iguape - nearest hotel: No hotel found
sesheke - nearest hotel: No hotel found
page - nearest hotel: No hotel found
gulf park estates - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
callao - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
tekeli - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
lapa - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
baijiantan - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
swan hill - nearest hotel: No hotel found
paphos - nearest hotel: No hotel found
ascension - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
arraial do cabo - neare

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,keflavik,64.0049,-22.5624,4.95,93,0,2.57,IS,1713742335,No hotel found
16,iguape,-24.7081,-47.5553,20.67,87,0,2.37,BR,1713742336,No hotel found
20,sesheke,-17.4759,24.2968,17.65,46,0,2.11,ZM,1713742340,No hotel found
30,page,36.9147,-111.4558,27.05,12,0,3.13,US,1713742350,No hotel found
33,gulf park estates,30.3919,-88.7611,21.01,43,0,7.20,US,1713742354,No hotel found
...,...,...,...,...,...,...,...,...,...,...
577,brookings,44.3114,-96.7984,15.21,21,0,5.66,US,1713742638,No hotel found
590,rustaq,23.3908,57.4244,25.62,52,0,0.77,OM,1713742679,No hotel found
591,marsh harbour,26.5412,-77.0636,24.39,72,0,6.27,BS,1713742577,No hotel found
592,chipinge,-20.1883,32.6236,13.71,68,0,1.51,ZW,1713742681,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)